# BMG Labtech CLARIOstar (Plus)

| Summary | Photo |
|---------|-------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"> <li><a href="https://www.bmglabtech.com/en/clariostar-plus/" target="_blank"><b>OEM Link</b></a></li> <li><b>Communication Protocol / Hardware:</b> Serial (FTDI) / USB</li> <li><b>Communication Level:</b> Firmware</li> <li><b>Measurement Modes:</b> Absorbance, Luminescence, Fluorescence</li> <li><b>Optical Systems:</b> Dual LVF Monochromator (320–840 nm), Physical Filters (240–900 nm), UV/Vis Spectrometer (220–1000 nm)</li> <li><b>Plate Delivery:</b> Drawer</li> <li><b>Additional Features:</b> Temperature control, Shaking, Configurable scan modes</li> <li>VID:PID <code>0403:BB68</code></li> </ul> | <div style="width:320px; text-align:center;"> ![clariostar](img/bmg-labtech-clariostar-plus.png) <br><i>Figure: BMG Labtech CLARIOstar Plus</i> </div> |

---
## Setup Instructions (Physical)

The CLARIOstar and CLARIOstar Plus require a minimum of two cable connections to be operational:
1. Power cord (standard IEC C13)
2. USB cable (USB-B with security screws at CLARIOstar end; USB-A at control PC end)

Optional:
If you have a plate stacking unit to use with the CLARIOstar (Plus), an additional RS-232 port is available on the CLARIOstar (Plus).

---
## Setup Instructions (Programmatic)

To control the BMG Labtech CLARIOstar (Plus), create a `PlateReader` frontend instance that uses a `CLARIOstarBackend` as its backend.

For convenience, store the backend as a separate variable so you can access CLARIOstar-specific features directly.

Set `protocol_mode` below to `"execution"` when connected to real hardware, or `"simulation"` to run the notebook end-to-end without a device.

In [1]:
from pylabrobot import verbose

verbose(True)

In [2]:
protocol_mode = "execution" # "execution" or "simulation"

In [3]:
import logging, os
from datetime import datetime
from pylabrobot.io import LOG_LEVEL_IO
ap_identifier = "test_protocol_1"
rounded_min = (datetime.now().minute // 10) * 10
ts = datetime.now().replace(minute=rounded_min, second=0, microsecond=0).strftime('%Y-%m-%d_%H-%M')

os.makedirs("./_logs", exist_ok=True)
fh = logging.FileHandler(f"./_logs/{ts}_{ap_identifier}_{protocol_mode}.log", mode="a")
fh.setLevel(LOG_LEVEL_IO)
fh.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(name)s - %(message)s"))

for name, lvl in [("pylabrobot", LOG_LEVEL_IO), ("manager", logging.DEBUG), ("device", logging.DEBUG)]:
    lgr = logging.getLogger(name)
    lgr.setLevel(lvl)
    if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename for h in lgr.handlers):
        lgr.addHandler(fh)

logger_manager = logging.getLogger("manager")
logger_manager.info(f"START AUTOMATED PROTOCOL\naP identifier: {ap_identifier}\n\nUser: Camillo\n")

In [4]:
from pylabrobot.plate_reading import PlateReader

if protocol_mode == "execution":
    from pylabrobot.plate_reading.bmg_labtech import CLARIOstarBackend
    clariostar_backend = CLARIOstarBackend()

elif protocol_mode == "simulation":
    from pylabrobot.plate_reading.bmg_labtech import CLARIOstarSimulatorBackend
    clariostar_backend = CLARIOstarSimulatorBackend()

pr = PlateReader(
    name="CLARIOstar",
    backend=clariostar_backend,
    size_x=0.0,
    size_y=0.0,
    size_z=0.0
)

If you have multiple FTDI devices connected, pass a `device_id` to the backend to select the correct one:

```python
clariostar_backend = CLARIOstarBackend(device_id="FT1234AB")
```

In [5]:
await pr.setup()

2026-02-19 11:38:48,353 - pylabrobot.io.ftdi - INFO - Successfully opened FTDI device: 430-2621
2026-02-19 11:38:48,612 - pylabrobot - INFO - read complete response: 476 bytes, 0201dc0c1325062600000100060546010302020900090d094c6001001eb0160d000220000e00032700000f000300000011000300000012000200001f00090100ec1ad2043a00212e00090104600003001eb4442f017845585f53503a313d6f70656e2c323d46696c74657220312c333d46696c74657220320d0a45585f4c503a313d6f70656e2c323d46696c74657220312c333d46696c74657220320d0a4253503a313d6f70656e2c323d46696c74657220312c333d46696c74657220322c343d46696c74657220330d0a454d5f53503a313d6f70656e2c323d46696c74657220312c333d46696c74657220320d0a454d5f4c503a313d6f70656e2c323d46696c74657220312c333d46696c74657220320d0a454d5f534c49543a313d6f70656e2c323d536c697420302e382c333d536c697420312e322c343d536c697420322e302c353d536c697420332e302c373d4650207061722c383d4650207065722c363d4c554d206c656e73650d0a45585f534c49543a313d6f70656e2c323d536c697420302e362c333d536c697420312e302c343d536c697420312e3

```{note}
Expected behaviour: the machine performs its initialization routine (FTDI connection, baudrate configuration, firmware initialization, and EEPROM data request).
```

---
## Defining a Plate

Every measurement requires a plate to be assigned to the plate reader. The backend dynamically encodes the plate geometry (dimensions, well positions, and well count) into the binary protocol — so you can use **any** PLR plate definition.

Here we use a standard Corning 96-well flat-bottom plate as an example:

In [6]:
from pylabrobot.resources.corning import Cor_96_wellplate_360ul_Fb

plate = Cor_96_wellplate_360ul_Fb("test_plate")
pr.assign_child_resource(plate)

---
## Query Machine


### EEPROM Machine Configuration Retrieval

During `setup()`, the backend queries the CLARIOstar's EEPROM (command `0x05 0x07`, 264 bytes) and firmware info (command `0x05 0x09`, 32 bytes). These responses are parsed into a `CLARIOstarConfig` dataclass.

#### What's decoded so far

| Field | Source | Status |
|-------|--------|--------|
| Model name, monochromator range, filter slots | Machine type code (EEPROM bytes 2-3) | Confirmed |
| Firmware version | Firmware info bytes 6-7 (uint16 BE / 1000) | Confirmed |
| Build date / time | Firmware info bytes 8-27 (null-terminated ASCII) | Confirmed |
| has_absorbance, has_fluorescence, has_luminescence, has_alpha | EEPROM bytes 11-14 | Confirmed |
| Serial number | FTDI chip (not in EEPROM) | Confirmed |
| has_pump1, has_pump2, has_stacker | Unknown offset | Needs 2nd unit with different options |


In [7]:
# View the parsed machine configuration
import dataclasses

config = clariostar_backend.get_machine_config()
if config is not None:
    for field in dataclasses.fields(config):
        print(f"  {field.name:25s} {getattr(config, field.name)}")
else:
    print("No config available (EEPROM not read yet)")

  serial_number             430-2621
  firmware_version          1.35
  firmware_build_timestamp  Nov 20 2020 11:51:21
  model_name                CLARIOstar Plus
  machine_type_code         38
  has_absorbance            True
  has_fluorescence          True
  has_luminescence          True
  has_alpha_technology      True
  has_pump1                 False
  has_pump2                 False
  has_stacker               False
  monochromator_range       (220, 1000)
  num_filter_slots          11


In [8]:
# View lifetime usage counters (queries the instrument each time — not cached)
counters = await clariostar_backend.request_usage_counters()
for name, value in counters.items():
    print(f"  {name:25s} {value:>12,}")

2026-02-19 11:39:00,246 - pylabrobot - INFO - read complete response: 50 bytes, 0200320c210500260000001d29d4000006c4000004ac0000039e0002628b000012b00000000a0000000a0000000a0005900d
2026-02-19 11:39:00,385 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000e00001320d
2026-02-19 11:39:00,388 - pylabrobot - INFO - status: {'valid': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}


  flashes                      1,911,252
  testruns                         1,732
  wells                          119,600
  well_movements                  92,600
  active_time_s                  156,299
  shake_time_s                     4,784
  pump1_usage                         10
  pump2_usage                         10
  alpha_time                          10


### Current Machine Status

The CLARIOstar reports its state as a dictionary of boolean status flags. You can query them at any time to check what the machine is doing.

The available flags are:

| Flag | Meaning | Shorthand |
|------|---------|-----------
| `standby` | Machine is in standby mode | |
| `valid` | Status response is valid | |
| `busy` | Machine is currently executing a command | `request_busy()` |
| `running` | A measurement run is in progress | |
| `unread_data` | Measurement data is available to read | |
| `initialized` | Instrument has been initialized | `request_initialization_status()` |
| `lid_open` | Top-front lid is open (filter and pump access) | |
| `drawer_open` | The drawer is open | `request_drawer_open()` |
| `plate_detected` | A plate is detected in the drawer | `request_plate_detected()` |
| `z_probed` | Z-height probing has completed | |
| `active` | Machine is active | |
| `filter_cover_open` | The filter cover is open | |

Flags without a shorthand can be accessed via `request_machine_status()["flag_name"]`.

In [9]:
status = await clariostar_backend.request_machine_status()
for flag, value in status.items():
    print(f"  {flag:20s} {value}")

2026-02-19 11:39:00,437 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000e00001320d


  standby              False
  valid                True
  busy                 False
  running              False
  unread_data          False
  initialized          True
  lid_open             False
  drawer_open          False
  plate_detected       True
  z_probed             True
  active               False
  filter_cover_open    False


In [10]:
# Convenience queries for common checks
if await clariostar_backend.request_plate_detected():
    # verify that the resource model is correct and that the plate is in the expected position
    print("Plate is in the drawer")
else:
    print("No plate detected")

if await clariostar_backend.request_busy():
    # verify that the machine is available for commands, i.e. not busy running a measurement
    print("Machine is busy")
else:
    print("Machine is ready")

2026-02-19 11:39:00,485 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000e00001320d
2026-02-19 11:39:00,525 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000e00001320d


Plate is in the drawer
Machine is ready


---
## Usage: Drawer

The CLARIOstar loads and unloads plates via a motorized drawer.

In [11]:
# # Open the drawer (plate out)
# await pr.open()

In [12]:
# # Place your plate on the drawer (manually or via a robotic arm), then close it (plate in)
# await pr.close()

---
## Usage: Temperature

The CLARIOstar has a built-in incubator that heats from both below and above the microplate. The bottom plate tracks the setpoint; the top plate targets setpoint + 0.5 °C to prevent condensation on the plate seal. Two temperature sensors (one per heating plate) are reported via the firmware.

Every measurement response also includes an embedded temperature (the `"temperature"` key in result dicts). This is the bottom plate sensor sampled at measurement start — it matches `measure_temperature("bottom")` at thermal equilibrium.

### Measuring temperature (no heating)

Use `measure_temperature()` to activate the sensors and read the current temperature without turning on the incubator. This sends a "monitor only" command, waits briefly for the sensors to report, and returns a single float.

Select which sensor to read with the `sensor` parameter: `"bottom"` (below the plate, default), `"top"` (above the plate), or `"mean"` (average of both).

In [13]:
# # Read current temperature (no heating) — defaults to bottom heating plate
# temp = await clariostar_backend.measure_temperature()
# print(f"Bottom plate: {temp:.1f} °C")

# # Read the top heating plate
# temp_top = await clariostar_backend.measure_temperature(sensor="top")
# print(f"Top plate:    {temp_top:.1f} °C")

# # Average of both
# temp_mean = await clariostar_backend.measure_temperature(sensor="mean")
# print(f"Mean:         {temp_mean:.1f} °C")

### Incubation (heating to a target temperature)

Use `start_temperature_control()` to turn on the incubator. The target is in °C (range: ~3 °C above ambient to 45 °C, in 0.1 °C steps). Use `measure_temperature()` to poll the current reading while heating.

In [14]:
# import asyncio

# # Heat to 37 °C
# await clariostar_backend.start_temperature_control(37.0)

# # Monitor the temperature as it climbs
# for _ in range(5):
#     temp = await clariostar_backend.measure_temperature()
#     print(f"Bottom plate: {temp:.1f} °C")
#     await asyncio.sleep(2)

# # Switch off incubator when done
# await clariostar_backend.stop_temperature_control()

In [15]:
logger_manager.info(f"\n========================\nSTART absorbance measurement - column 1 - 600 nm - OD\n")
# Read only column 1 for faster testing
column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]

results = await pr.read_absorbance(
    wavelength=600,
    wells=column_1_wells,
    use_new_return_type=True,
    report="OD",
    well_scan="point",
    wait=True
)

print(f"OD600 of well A1: {results[0]['data'][0][0]}")
print(f"Temperature: {results[0]['temperature']} \u00b0C")
print(results)
logger_manager.info(f"\nEND absorbance measurement - column 1 - 600 nm - OD\n============================")

2026-02-19 11:39:00,681 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000e00001320d
2026-02-19 11:39:00,780 - pylabrobot - INFO - read complete response: 53 bytes, 0200350c03250426000000004e2000000018010000000d00000001010000000000000001000000030001000000000000220001520d
2026-02-19 11:39:00,782 - pylabrobot - INFO - Run command accepted: total_values=24, status=250426
2026-02-19 11:39:00,939 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400000000e000015f0d
2026-02-19 11:39:01,505 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:39:02,508 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:39:03,074 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:39:04,076 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:39:24,579 - pylabrobot - WARNING

OD600 of well A1: 0.07863802578648804
Temperature: 23.5 °C
[{'wavelength': 600, 'data': [[0.07863802578648804, None, None, None, None, None, None, None, None, None, None, None], [0.08954786709047734, None, None, None, None, None, None, None, None, None, None, None], [0.0930677456334815, None, None, None, None, None, None, None, None, None, None, None], [0.08600835341243697, None, None, None, None, None, None, None, None, None, None, None], [0.09739939640247092, None, None, None, None, None, None, None, None, None, None, None], [0.15940212854355917, None, None, None, None, None, None, None, None, None, None, None], [0.5554107690594093, None, None, None, None, None, None, None, None, None, None, None], [2.3356375476532194, None, None, None, None, None, None, None, None, None, None, None]], 'temperature': 23.5, 'time': 1771501164.6739452}]


In [16]:
import pandas as pd

pd.DataFrame(results[0]['data'])


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.078638,None,None,None,None,None,None,None,None,None,None,None
1,0.089548,None,None,None,None,None,None,None,None,None,None,None
2,0.093068,None,None,None,None,None,None,None,None,None,None,None
3,0.086008,None,None,None,None,None,None,None,None,None,None,None
4,0.097399,None,None,None,None,None,None,None,None,None,None,None
5,0.159402,None,None,None,None,None,None,None,None,None,None,None
6,0.555411,None,None,None,None,None,None,None,None,None,None,None
7,2.335638,None,None,None,None,None,None,None,None,None,None,None


---
## Usage: Measuring Absorbance

Absorbance measures how much light a sample absorbs at a given wavelength.
Results can be reported as optical density (OD) or percent transmittance.

**Transmittance** is the percentage of light that passes through the sample. A clear well lets ~100% through; a dense culture might let only 10% through.

**OD (optical density)** is the log-transformed version: `OD = log10(100 / T%)`. The log scale is useful because it is proportional to the concentration of the absorbing substance (Beer-Lambert law), whereas transmittance is not. Most protocols report OD.

| Transmittance | OD | Meaning |
|--------------:|---:|---------|
| 100% | 0.0 | No absorption (blank / empty well) |
| 10% | 1.0 | 90% of light absorbed |
| 1% | 2.0 | 99% of light absorbed |

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `wavelength` | `int` | *required* | Wavelength in nm (220–1000) |
| `report` | `"OD"`, `"transmittance"`, or `"raw"` | `"OD"` | Output format |
| `wavelengths` | `List[int]` | — | Multiple wavelengths (1–8), overrides `wavelength` |
| `flashes_per_well` | `int` | `5` | Xenon lamp flashes per well (1–200). OEM default. Values ≥ 5 give converged OD; each extra flash adds ~12 ms/well. |
| `pause_time_per_well` | `int` | `0` | Per-well positioning delay in deciseconds (0–10). Adds ~0.1 s/well/unit. Useful after shaking. |
| `settling_time_before_measurement` | `int` | `0` | Once-per-run post-shake settling delay in seconds. Adds ~1.1 s/unit. No effect on accuracy. |

For a detailed explanation of how the CLARIOstar converts raw detector counts to OD values, see [Reference: How Absorbance Calculation Works](#reference-how-absorbance-calculation-works) at the end of this notebook.

### Single-wavelength absorbance (OD)

In [17]:
import time, asyncio, statistics

# 16-well pattern across columns 1-3 (includes A1 and H1)
# Col 1: all 8 rows | Col 2: A,C,E,G (even rows) | Col 3: B,D,F,H (odd rows)
test_wells_16 = (
    [plate.get_well(f"{row}1") for row in "ABCDEFGH"]
    + [plate.get_well(f"{row}2") for row in "ACEG"]
    + [plate.get_well(f"{row}3") for row in "BDFH"]
)

# (row_index, col_index, label) for the 16 wells in the 8x12 grid
_W16 = (
    [(r, 0, f"{ch}1") for r, ch in enumerate("ABCDEFGH")]
    + [(r, 1, f"{ch}2") for r, ch in zip([0, 2, 4, 6], "ACEG")]
    + [(r, 2, f"{ch}3") for r, ch in zip([1, 3, 5, 7], "BDFH")]
)

def fmt16(results, label):
    """Print all 16 well OD values for a single measurement."""
    d, t = results[0]["data"], results[0]["temperature"]
    v = [f"{n}={d[r][c]:.4f}" if d[r][c] is not None else f"{n}=None"
         for r, c, n in _W16]
    print(f"  {label}  T={t:.1f}\u00b0C")
    print(f"    {' '.join(v[:8])}")
    print(f"    {' '.join(v[8:12])}")
    print(f"    {' '.join(v[12:])}")

def summarize(a1_vals, h1_vals, timings):
    """Print replicate summary statistics for A1, H1, and timing."""
    print(f"  -- A1: mean={statistics.mean(a1_vals):.4f}  std={statistics.stdev(a1_vals):.5f}")
    print(f"  -- H1: mean={statistics.mean(h1_vals):.4f}  std={statistics.stdev(h1_vals):.5f}")
    print(f"  -- Time: mean={statistics.mean(timings):.1f}s  std={statistics.stdev(timings):.2f}s\n")

print(f"Selected {len(test_wells_16)} wells: {', '.join(w.name for w in test_wells_16)}")

Selected 16 wells: test_plate_well_A1, test_plate_well_B1, test_plate_well_C1, test_plate_well_D1, test_plate_well_E1, test_plate_well_F1, test_plate_well_G1, test_plate_well_H1, test_plate_well_A2, test_plate_well_C2, test_plate_well_E2, test_plate_well_G2, test_plate_well_B3, test_plate_well_D3, test_plate_well_F3, test_plate_well_H3


---
### Parameter Exploration: pause_time_per_well, settling_time_before_measurement, flashes_per_well

Comprehensive characterization of firmware measurement parameters. Each test uses
**16 wells** across columns 1-3 (staggered pattern: all 8 rows in col 1, alternating
rows in cols 2-3; A1 and H1 always included) at 600 nm.

**Test matrix per parameter:**
- 3 replicates per value at ambient temperature, `vertical=False` (default scan direction)
- Vertical scan comparison: `vertical=True` vs `vertical=False` for selected values
- Temperature comparison: selected values at 29°C incubation vs ambient

All heated tests are grouped in cell 37 to avoid temperature cycling (CLARIOstar has
no active cooling — heating once then running all 29°C tests is ~15 min faster than
interleaving ambient and heated conditions).

#### Previous Results (8 wells, column 1 only)

**`pause_time_per_well`** — per-well positioning delay (deciseconds, 0-10)

| pause_time_per_well | Elapsed (run 1) | Elapsed (run 2) | A1 OD |
|:---:|---:|---:|---:|
| 0 | 8.9 s | 7.0 s | 0.078 |
| 1 | 9.4 s | 8.1 s | 0.078 |
| 5 | 13.5 s | 11.7 s | 0.077 |
| 10 | 17.6 s | 15.8 s | 0.076 |

Adds ~0.11 s per well per decisecond (~0.88 s total per unit for 8 wells). No meaningful effect on OD accuracy (~2% spread, within replicate noise). Useful after shaking to let vibrations dampen.

**`settling_time_before_measurement`** — once-per-run post-shake settling delay (seconds)

| settling_time_before_measurement | Elapsed (run 1) | Elapsed (run 2) | A1 OD |
|:---:|---:|---:|---:|
| 0 | 8.7 s | 7.0 s | 0.078 |
| 1 | 9.9 s | 8.1 s | 0.079 |
| 2 | 11.1 s | 9.3 s | 0.078 |
| 5 | 29.4 s* | 12.3 s | 0.078 |

\* Transient firmware delay (see note below). Linear scaling confirmed: ~1.1 s/unit.

**`flashes_per_well`** — xenon lamp flashes averaged per well (1-200)

| flashes_per_well | Elapsed (run 1) | Elapsed (run 2) | A1 OD |
|:---:|---:|---:|---:|
| 1 | 20.2 s* | 7.0 s | 0.0835 |
| 5 | 7.0 s | 20.9 s* | 0.078 |
| 22 | 9.1 s | 8.7 s | 0.078 |
| 50 | 11.7 s | 11.7 s | 0.077 |
| 100 | 16.4 s | 16.4 s | 0.077 |

\* Transient firmware delay. fpw=1 gives reduced OD accuracy (+7%) in both runs — **avoid for quantitative work**. OEM defaults (5 for point, 7 for orbital) are the sweet spot.

```{note}
**Transient firmware delay (~14 s)**: A one-time delay of ~14 s occurs once per session at an unpredictable measurement position. In run 1 it hit `settling_time_before_measurement=5` and `flashes_per_well=1`; in run 2 it hit `flashes_per_well=5` instead. This is likely a firmware-level lamp calibration or thermal stabilization event. It does not affect measurement accuracy, only timing. Excluding the transient, all timing parameters scale linearly and reproducibly across runs.
```

In [18]:
# --- Test 1: pause_time_per_well ---
# Per-well positioning delay (deciseconds). The optics head moves to a well,
# waits this long, then fires the xenon flash.
# Test: 3 replicates per value + vertical scan comparison.
# 16 wells across cols 1-3.

print("=" * 72)
print("pause_time_per_well: 3 replicates, ambient, vertical=False")
print("=" * 72)
for ptpw in [0, 1, 5, 10]:
    a1_v, h1_v, t_v = [], [], []
    for rep in range(3):
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            pause_time_per_well=ptpw,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        t_v.append(elapsed)
        a1_v.append(results[0]["data"][0][0])
        h1_v.append(results[0]["data"][7][0])
        fmt16(results, f"ptpw={ptpw:2d} rep={rep+1} {elapsed:5.1f}s")
    summarize(a1_v, h1_v, t_v)

print("=" * 72)
print("pause_time_per_well: vertical=True vs vertical=False")
print("=" * 72)
for ptpw in [0, 5]:
    for vert in [False, True]:
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            pause_time_per_well=ptpw, vertical=vert,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        fmt16(results, f"ptpw={ptpw:2d} vertical={str(vert):5s} {elapsed:5.1f}s")
    print()

2026-02-19 11:39:25,210 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010506260000000000000000000000e00001380d
2026-02-19 11:39:25,304 - pylabrobot - INFO - read complete response: 53 bytes, 0200350c03250426000000002ee000000028010000000f000000010100000000000000010000000500010000000000003800021c0d
2026-02-19 11:39:25,306 - pylabrobot - INFO - Run command accepted: total_values=40, status=250426


pause_time_per_well: 3 replicates, ambient, vertical=False


2026-02-19 11:39:25,461 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400000000e000015f0d
2026-02-19 11:39:26,026 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:39:27,028 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:39:27,582 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:39:28,586 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:39:29,125 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:39:29,165 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d50000000000eb00f2e00004180d
2026-02-19 11:39:29,166 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

  ptpw= 0 rep=1   9.4s  T=23.5°C
    A1=0.0788 B1=0.0868 C1=0.0906 D1=0.0883 E1=0.0811 F1=0.0871 G1=0.0830 H1=0.0867
    A2=0.1010 C2=0.0879 E2=0.1653 G2=0.0902
    B3=0.5627 D3=0.0865 F3=2.4095 H3=0.0881


2026-02-19 11:39:34,818 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:39:35,385 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:39:36,388 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:39:36,948 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:39:37,952 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:39:38,493 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:39:38,533 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d50000000000eb00f2e00004180d
2026-02-19 11:39:38,535 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

...

  ptpw= 0 vertical=True    9.4s  T=23.5°C
    A1=0.0786 B1=0.0851 C1=0.0880 D1=0.0880 E1=0.0904 F1=0.0870 G1=0.0863 H1=0.0879
    A2=0.0969 C2=0.0874 E2=0.1618 G2=0.0898
    B3=0.5591 D3=0.0867 F3=2.3762 H3=0.0877



2026-02-19 11:43:13,202 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:43:13,203 - pylabrobot - INFO - read complete response: 21 bytes, 0200180c01a504260000fa0500000000eb00f2e00d
2026-02-19 11:43:13,204 - pylabrobot - WARNING - Could not unframe status response: 0200180c01a504260000fa0500000000eb00f2e00d
2026-02-19 11:43:13,744 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010507260000000000000000eb00f2e00003160d
2026-02-19 11:43:13,745 - pylabrobot - INFO - BUSY cleared, measurement complete
2026-02-19 11:43:13,798 - pylabrobot - INFO - read complete response: 188 bytes, 0200bc0c020506260000a900240024001dffe20200010001001001000100000001010000000000eb0031fb9b0030e6fa0031889000309953003177c60030a6dd0031a7d50030bb85003079e20030b0010029b8a800307b96001099f00030ced200004452003096e400008da200008dc100008d3500008da100008d7800008d5a00008dd900008d7500008d9e00008d7200008dbe00008db800008d7100008d9800008d8500008d8d003ba28c00008b0d00008da7000000000031470d
2026-0

  ptpw= 5 vertical=False  20.8s  T=23.5°C
    A1=0.0766 B1=0.0865 C1=0.0792 D1=0.0888 E1=0.0806 F1=0.0875 G1=0.0801 H1=0.0871
    A2=0.0899 C2=0.0874 E2=0.1554 G2=0.0901
    B3=0.5547 D3=0.0868 F3=2.3488 H3=0.0886


2026-02-19 11:43:14,093 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400000000e000015f0d
2026-02-19 11:43:14,659 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:15,663 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:43:16,224 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:17,228 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:43:17,768 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:17,810 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d50000000000eb00f2e00004180d
2026-02-19 11:43:17,811 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

  ptpw= 5 vertical=True   17.7s  T=23.5°C
    A1=0.0777 B1=0.0853 C1=0.0846 D1=0.0877 E1=0.0807 F1=0.0873 G1=0.0794 H1=0.0884
    A2=0.0893 C2=0.0873 E2=0.1553 G2=0.0909
    B3=0.5551 D3=0.0868 F3=2.3414 H3=0.0885



In [19]:
# --- Test 2: settling_time_before_measurement ---
# Once-per-run delay before measurement starts (seconds). Encoded as flag byte +
# uint16 BE seconds. Primarily useful as a post-shake settling delay.
# Test: 3 replicates per value + vertical scan comparison.
# 16 wells across cols 1-3.

print("=" * 72)
print("settling_time_before_measurement: 3 replicates, ambient, vertical=False")
print("=" * 72)
for stbm in [0, 1, 2, 5]:
    a1_v, h1_v, t_v = [], [], []
    for rep in range(3):
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            settling_time_before_measurement=stbm,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        t_v.append(elapsed)
        a1_v.append(results[0]["data"][0][0])
        h1_v.append(results[0]["data"][7][0])
        fmt16(results, f"stbm={stbm:2d} rep={rep+1} {elapsed:5.1f}s")
    summarize(a1_v, h1_v, t_v)

print("=" * 72)
print("settling_time_before_measurement: vertical=True vs vertical=False")
print("=" * 72)
for stbm in [0, 5]:
    for vert in [False, True]:
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            settling_time_before_measurement=stbm, vertical=vert,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        fmt16(results, f"stbm={stbm:2d} vertical={str(vert):5s} {elapsed:5.1f}s")
    print()

2026-02-19 11:43:31,525 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010506260000000000000000000000e00001380d
2026-02-19 11:43:31,628 - pylabrobot - INFO - read complete response: 53 bytes, 0200350c03250426000000002ee000000028010000000f000000010100000000000000010000000500010000000000003800021c0d
2026-02-19 11:43:31,630 - pylabrobot - INFO - Run command accepted: total_values=40, status=250426


settling_time_before_measurement: 3 replicates, ambient, vertical=False


2026-02-19 11:43:31,787 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:32,353 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:33,357 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:43:33,916 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:34,918 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:43:35,460 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:35,500 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d50000000000eb00f2e00004180d
2026-02-19 11:43:35,502 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

  stbm= 0 rep=1   9.4s  T=23.5°C
    A1=0.0785 B1=0.0864 C1=0.0895 D1=0.0882 E1=0.0815 F1=0.0868 G1=0.0833 H1=0.0864
    A2=0.1013 C2=0.0873 E2=0.1674 G2=0.0903
    B3=0.5662 D3=0.0865 F3=2.3415 H3=0.0876


2026-02-19 11:43:41,165 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:41,731 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:42,734 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:43:43,295 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:44,299 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:43:44,837 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:43:44,875 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d50000000000eb00f2e00004180d
2026-02-19 11:43:44,876 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

...

  stbm= 0 vertical=True   10.9s  T=23.5°C
    A1=0.0789 B1=0.0865 C1=0.0886 D1=0.0896 E1=0.0923 F1=0.0872 G1=0.0882 H1=0.0877
    A2=0.0976 C2=0.0878 E2=0.1630 G2=0.0907
    B3=0.5613 D3=0.0877 F3=2.3563 H3=0.0885



2026-02-19 11:46:12,179 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:12,744 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:13,748 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:46:14,309 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:15,313 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:46:15,851 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:15,889 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d50000000000eb00f2e00004180d
2026-02-19 11:46:15,891 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

  stbm= 5 vertical=False  14.1s  T=23.5°C
    A1=0.0783 B1=0.0870 C1=0.0893 D1=0.0886 E1=0.0822 F1=0.0874 G1=0.0838 H1=0.0878
    A2=0.1025 C2=0.0877 E2=0.1695 G2=0.0910
    B3=0.5666 D3=0.0866 F3=2.3446 H3=0.0885


2026-02-19 11:46:26,290 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:26,856 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:27,858 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:46:28,420 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:29,424 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:46:29,964 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:30,004 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d50000000000eb00f2e00004180d
2026-02-19 11:46:30,005 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

  stbm= 5 vertical=True   14.7s  T=23.5°C
    A1=0.0787 B1=0.0865 C1=0.0875 D1=0.0898 E1=0.0921 F1=0.0868 G1=0.0888 H1=0.0875
    A2=0.0989 C2=0.0881 E2=0.1628 G2=0.0914
    B3=0.5607 D3=0.0879 F3=2.3466 H3=0.0891



In [20]:
# --- Test 3: flashes_per_well ---
# Xenon lamp flashes averaged per well (1-200). More flashes = better SNR but slower.
# Test: 3 replicates per value + vertical scan comparison.
# 16 wells across cols 1-3.

print("=" * 72)
print("flashes_per_well: 3 replicates, ambient, vertical=False")
print("=" * 72)
for fpw in [1, 5, 22, 50, 100]:
    a1_v, h1_v, t_v = [], [], []
    for rep in range(3):
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            flashes_per_well=fpw,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        t_v.append(elapsed)
        a1_v.append(results[0]["data"][0][0])
        h1_v.append(results[0]["data"][7][0])
        fmt16(results, f"fpw={fpw:3d} rep={rep+1} {elapsed:5.1f}s")
    summarize(a1_v, h1_v, t_v)

print("=" * 72)
print("flashes_per_well: vertical=True vs vertical=False")
print("=" * 72)
for fpw in [5, 50]:
    for vert in [False, True]:
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            flashes_per_well=fpw, vertical=vert,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        fmt16(results, f"fpw={fpw:3d} vertical={str(vert):5s} {elapsed:5.1f}s")
    print()

# ============================================================
# 29 deg C INCUBATION COMPARISON - all three parameters
# CLARIOstar has no active cooling, so we heat once, run all
# heated tests, then stop. Subset of values to keep it practical.
# ============================================================
print("=" * 72)
print("29\u00b0C INCUBATION COMPARISON")
print("=" * 72)
await clariostar_backend.start_temperature_control(29.0)
for attempt in range(18):  # up to 3 minutes
    temp = await clariostar_backend.measure_temperature()
    print(f"  Stabilizing... {temp:.1f}\u00b0C")
    if abs(temp - 29.0) < 0.5:
        print(f"  Target reached: {temp:.1f}\u00b0C")
        break
    await asyncio.sleep(10)

print("\npause_time_per_well at 29\u00b0C:")
for ptpw in [0, 5]:
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=test_wells_16,
        pause_time_per_well=ptpw,
        use_new_return_type=True,
    )
    elapsed = time.time() - t0
    fmt16(results, f"ptpw={ptpw:2d} @29\u00b0C {elapsed:5.1f}s")

print("\nsettling_time_before_measurement at 29\u00b0C:")
for stbm in [0, 5]:
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=test_wells_16,
        settling_time_before_measurement=stbm,
        use_new_return_type=True,
    )
    elapsed = time.time() - t0
    fmt16(results, f"stbm={stbm:2d} @29\u00b0C {elapsed:5.1f}s")

print("\nflashes_per_well at 29\u00b0C:")
for fpw in [5, 50]:
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=test_wells_16,
        flashes_per_well=fpw,
        use_new_return_type=True,
    )
    elapsed = time.time() - t0
    fmt16(results, f"fpw={fpw:3d} @29\u00b0C {elapsed:5.1f}s")

await clariostar_backend.stop_temperature_control()
print("\nTemperature control stopped.")

2026-02-19 11:46:40,766 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010506260000000000000000000000e00001380d
2026-02-19 11:46:40,865 - pylabrobot - INFO - read complete response: 53 bytes, 0200350c03250426000000002ee000000028010000000f000000010100000000000000010000000500010000000000000b0001ef0d
2026-02-19 11:46:40,868 - pylabrobot - INFO - Run command accepted: total_values=40, status=250426


flashes_per_well: 3 replicates, ambient, vertical=False


2026-02-19 11:46:41,024 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400000000e000015f0d
2026-02-19 11:46:41,590 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:42,594 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:46:43,154 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:44,156 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:46:44,694 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:44,732 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d50000000000eb00f2e00004180d
2026-02-19 11:46:44,733 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

  fpw=  1 rep=1   8.2s  T=23.5°C
    A1=0.0826 B1=0.0856 C1=0.0955 D1=0.0885 E1=0.0845 F1=0.0867 G1=0.0923 H1=0.0870
    A2=0.1332 C2=0.0887 E2=0.2101 G2=0.0896
    B3=0.6086 D3=0.0868 F3=2.2594 H3=0.0875


2026-02-19 11:46:49,226 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c01a504260000fa0500000000eb00f2e00004b20d
2026-02-19 11:46:49,792 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:50,796 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:46:51,356 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:52,358 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:46:52,896 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400eb00f2e000033c0d
2026-02-19 11:46:52,936 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d50000000000eb00f2e00004180d
2026-02-19 11:46:52,937 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

...

  stbm= 5 @29°C  13.9s  T=28.6°C
    A1=0.0785 B1=0.0868 C1=0.0900 D1=0.0893 E1=0.0817 F1=0.0875 G1=0.0863 H1=0.0871
    A2=0.1044 C2=0.0873 E2=0.1693 G2=0.0900
    B3=0.5669 D3=0.0873 F3=2.3655 H3=0.0881

flashes_per_well at 29°C:


2026-02-19 11:53:43,910 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c01a504260000fa05000000011d0121c00002f50d
2026-02-19 11:53:44,476 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c0125042600000401000004011d0121c000017f0d
2026-02-19 11:53:45,480 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:53:46,040 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c0125042600000401000004011d0121c000017f0d
2026-02-19 11:53:47,042 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:53:47,580 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c0125042600000401000004011c0121c000017e0d
2026-02-19 11:53:47,618 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d500000000011c0121c000025a0d
2026-02-19 11:53:47,619 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

  fpw=  5 @29°C   9.4s  T=28.4°C
    A1=0.0786 B1=0.0866 C1=0.0900 D1=0.0892 E1=0.0818 F1=0.0873 G1=0.0864 H1=0.0872
    A2=0.1042 C2=0.0876 E2=0.1677 G2=0.0902
    B3=0.5671 D3=0.0872 F3=2.3278 H3=0.0887


2026-02-19 11:53:53,274 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c0125042600000401000004011c0121c000017e0d
2026-02-19 11:53:53,841 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c0125042600000401000004011c0121c000017e0d
2026-02-19 11:53:54,843 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:53:55,404 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c0125042600000401000004011c0120c000017d0d
2026-02-19 11:53:56,406 - pylabrobot - WARNING - timed out reading response
2026-02-19 11:53:56,944 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c0125042600000401000004011c0120c000017d0d
2026-02-19 11:53:56,984 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c013504260000d500000000011c0120c00002590d
2026-02-19 11:53:56,986 - pylabrobot - INFO - Received unsolicited status notification: {'valid': True, 'busy': True, 'running': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-

  fpw= 50 @29°C  32.4s  T=28.4°C
    A1=0.0773 B1=0.0865 C1=0.0829 D1=0.0895 E1=0.0809 F1=0.0874 G1=0.0843 H1=0.0870
    A2=0.0927 C2=0.0879 E2=0.1581 G2=0.0907
    B3=0.5586 D3=0.0874 F3=2.3712 H3=0.0885

Temperature control stopped.


In [ ]:
await pr.stop()

### Absorbance as transmittance

In [ ]:
results = await pr.read_absorbance(
    wavelength=450,
    report="transmittance",
    use_new_return_type=True
s)

# Transmittance is in percent (0-100)
print(f"Transmittance at 450nm, well A1: {results[0]['data'][0][0]}%")

### Multi-wavelength absorbance

Read up to 8 wavelengths in a single run. This is faster than running separate measurements because the plate is only scanned once.

Pass `wavelengths` as a backend keyword argument (this overrides the `wavelength` parameter):

In [ ]:
results = await pr.read_absorbance(
    wavelength=600,             # ignored when wavelengths is provided
    wavelengths=[260, 280, 450, 600, 750],
    use_new_return_type=True
)

# One result dict per wavelength
for r in results:
    print(f"Wavelength {r['wavelength']} nm -> A1 OD: {r['data'][0][0]}")

---
## Usage: Measuring Fluorescence

Fluorescence measures light emitted by a fluorophore after excitation at a specific wavelength.

### Optical paths

The CLARIOstar has three optical systems on the same mechanical rail. PyLabRobot currently uses the **monochromator path** for fluorescence. Here is how the three systems compare:

| Optical System | Spectral Range | Bandwidth | Used By PLR | Best For |
|----------------|---------------|-----------|:-----------:|----------|
| **Dual LVF Monochromator** | 320–840 nm | 8–100 nm (software selectable) | Yes | General fluorescence (GFP, mCherry, DAPI, etc.) |
| **Physical Filters** (11 slots) | 240–900 nm | Fixed per filter | Not yet | FP, HTRF, AlphaScreen, TR-FRET |
| **UV/Vis Spectrometer** | 220–1000 nm | 3 nm fixed | Yes (absorbance) | Absorbance only |

The monochromator offers freely tunable wavelengths and bandwidths, which is sufficient for most fluorescence assays. Physical filter slots (up to 4 excitation, 3 dichroic, 4 emission) provide higher sensitivity for specialized applications — see [Planned Features](#planned-features) for filter-based fluorescence support.

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `excitation_wavelength` | `int` | *required* | Excitation center wavelength in nm |
| `emission_wavelength` | `int` | *required* | Emission center wavelength in nm |
| `focal_height` | `float` | *required* | Focal height in mm (0–25) |
| `gain` | `int` | `1000` | Detector gain |
| `ex_bandwidth` | `int` | `20` | Excitation bandwidth in nm |
| `em_bandwidth` | `int` | `40` | Emission bandwidth in nm |
| `dichroic` | `int` | auto | Dichroic wavelength × 10 (auto-calculated as `(ex + em) * 5`) |
| `flashes` | `int` | `100` | Number of flashes per well (0–200) |
| `pause_time_per_well` | `int` | `0` | Per-well pause in deciseconds (0–10) |
| `settling_time_before_measurement` | `int` | `0` | Once-per-run post-shake settling delay in seconds |
| `bottom_optic` | `bool` | `False` | Read from bottom instead of top |

### Basic fluorescence read (e.g. GFP: ex 485 / em 528)

In [ ]:
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    use_new_return_type=True
)

# results is a list with one dict containing:
#   "ex_wavelength": int, "em_wavelength": int,
#   "data": 8x12 grid, "temperature": float, "time": float
print(f"GFP signal at A1: {results[0]['data'][0][0]}")
print(f"Temperature: {results[0]['temperature']} \u00b0C")

### Fluorescence with custom gain, bandwidth, and flashes

In [ ]:
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    gain=1500,           # increase gain for weak signals
    ex_bandwidth=10,     # narrow excitation bandwidth for better specificity
    em_bandwidth=20,     # narrow emission bandwidth
    flashes=150,         # more flashes for better signal-to-noise
    use_new_return_type=True
)

### Bottom-optic fluorescence

Use bottom-optic reading when working with cell monolayers, adherent cells, or clear-bottom plates where top-reading would be affected by the meniscus or lid:

In [ ]:
results = await pr.read_fluorescence(
    excitation_wavelength=544,
    emission_wavelength=590,
    focal_height=4.5,
    bottom_optic=True,
    use_new_return_type=True
)

---
## Usage: Measuring Luminescence

Luminescence measures light emitted by a chemical or biological reaction (no excitation light source).

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `focal_height` | `float` | `13` | Focal height in mm (0–25) |

### Basic luminescence read (full plate)

In [9]:
results = await pr.read_luminescence(
    focal_height=13.0,
    use_new_return_type=True
)

# results is a list of dicts, each containing:
#   "data": 8x12 grid of float values (rows x columns)
#   "temperature": float (Celsius)
#   "time": float (unix timestamp)
print(f"Temperature: {results[0]['temperature']} \u00b0C")
print(f"Well A1 value: {results[0]['data'][0][0]}")

Temperature: 26.0 °C
Well A1 value: 3891.5845732141615


---
## Usage: Partial Well Selection

All three measurement modes (luminescence, absorbance, fluorescence) support reading a subset of wells instead of the full plate. This can significantly speed up reads when you only need data from specific wells.

Unread wells are filled with `None` in the output grid.

In [ ]:
# Read only column 1 (wells A1-H1)
column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]

results = await pr.read_absorbance(
    wavelength=600,
    wells=column_1_wells,
    use_new_return_type=True
)

# Only column 1 has values; all other wells are None
for row_idx, row_letter in enumerate("ABCDEFGH"):
    val = results[0]['data'][row_idx][0]  # column 0 = column 1
    print(f"Well {row_letter}1: OD = {val}")

In [ ]:
# Read specific wells by name
selected_wells = [plate.get_well("A1"), plate.get_well("D6"), plate.get_well("H12")]

results = await pr.read_luminescence(
    focal_height=13.0,
    wells=selected_wells,
    use_new_return_type=True
)

In [ ]:
# Partial well selection works for fluorescence too
row_A_wells = [plate.get_well(f"A{col}") for col in range(1, 13)]

results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    wells=row_A_wells,
    use_new_return_type=True
)

---
## Usage: Scan Mode Configuration

The scan mode controls how the plate reader traverses the plate during a measurement. Configurable via backend keyword arguments on all three read methods.

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `start_corner` | `StartCorner` | `TOP_LEFT` | Which corner to begin reading from |
| `unidirectional` | `bool` | `False` (lum/fl), `True` (abs) | Read in one direction only (vs. bidirectional zigzag) |
| `vertical` | `bool` | `False` | Read columns instead of rows |
| `flying_mode` | `bool` | `False` | Continuous movement (fluorescence only, max 3 flashes) |

Available start corners:
- `StartCorner.TOP_LEFT` (default)
- `StartCorner.TOP_RIGHT`
- `StartCorner.BOTTOM_LEFT`
- `StartCorner.BOTTOM_RIGHT`

In [ ]:
from pylabrobot.plate_reading.bmg_labtech.clariostar_backend import StartCorner

In [ ]:
# Read from bottom-right corner, vertical scan, unidirectional
results = await pr.read_absorbance(
    wavelength=600,
    start_corner=StartCorner.BOTTOM_RIGHT,
    unidirectional=True,
    vertical=True,
    use_new_return_type=True
)

In [ ]:
# Flying mode for fast fluorescence reads (max 3 flashes)
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    flying_mode=True,
    flashes=3,
    start_corner=StartCorner.TOP_LEFT,
    use_new_return_type=True
)

```{note}
Flying mode keeps the optics head moving continuously instead of stopping at each well. This is much faster but limits you to a maximum of 3 flashes per well. Only available for fluorescence.
```

---
## Usage: Pre-Measurement Shaking

The CLARIOstar has a built-in shaker. When configured as part of a measurement call, the plate is shaken **immediately before** the optics head begins reading. This is useful for resuspending cells or mixing reagents right before a read.

```{note}
Currently, shaking is only available as a step embedded in a measurement run. Standalone shaking (shake without measuring) is a planned feature — see [Planned Features](#planned-features) below.
```

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `shake_type` | `ShakerType` | `ORBITAL` | Type of shaking motion |
| `shake_speed_rpm` | `int` | `0` | Speed in RPM (100–700, in steps of 100) |
| `shake_duration_s` | `int` | `0` | Duration in seconds (0 = no shaking) |

Available shaker types:

| Type | Description | Max RPM |
|------|-------------|--------:|
| `ShakerType.ORBITAL` | Circular orbit | 700 |
| `ShakerType.LINEAR` | Back and forth in a line | 700 |
| `ShakerType.DOUBLE_ORBITAL` | Figure-eight pattern | 700 |
| `ShakerType.MEANDER` | Meandering path | **300** |

In [ ]:
from pylabrobot.plate_reading.bmg_labtech.clariostar_backend import ShakerType

In [ ]:
# Orbital shake at 300 RPM for 5 seconds before reading luminescence
results = await pr.read_luminescence(
    focal_height=13.0,
    shake_type=ShakerType.ORBITAL,
    shake_speed_rpm=300,
    shake_duration_s=5,
    use_new_return_type=True
)

In [ ]:
# Linear shake at 500 RPM for 10 seconds before reading absorbance
results = await pr.read_absorbance(
    wavelength=600,
    shake_type=ShakerType.LINEAR,
    shake_speed_rpm=500,
    shake_duration_s=10,
    use_new_return_type=True
)

In [ ]:
# Double orbital shake before fluorescence
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    shake_type=ShakerType.DOUBLE_ORBITAL,
    shake_speed_rpm=400,
    shake_duration_s=3,
    use_new_return_type=True
)

```{warning}
The `MEANDER` shaker type is limited to a maximum of 300 RPM. Exceeding this will raise a `ValueError`.
```

---
## Usage: Combining Features

All features can be combined in a single read call. Here is an example that uses partial well selection, pre-measurement shaking, and a custom scan configuration together:

In [ ]:
# Read fluorescence from wells A1-A12 with orbital shake, bottom-right start, and custom gain
row_A = [plate.get_well(f"A{col}") for col in range(1, 13)]

results = await pr.read_fluorescence(
    excitation_wavelength=544,
    emission_wavelength=590,
    focal_height=6.0,
    wells=row_A,
    gain=2000,
    flashes=50,
    bottom_optic=True,
    shake_type=ShakerType.ORBITAL,
    shake_speed_rpm=300,
    shake_duration_s=5,
    start_corner=StartCorner.BOTTOM_RIGHT,
    unidirectional=True,
    use_new_return_type=True
)

# Print row A results
for col_idx in range(12):
    val = results[0]['data'][0][col_idx]
    print(f"A{col_idx+1}: {val}")

---
## Usage: Non-Blocking Reads

By default, every `read_*` call blocks until the measurement is complete. For long-running operations (e.g. a 384-well plate with multiple wavelengths can take 10–15 minutes), you can start the measurement without waiting, do other work, and collect the results later.

Pass `wait=False` to any read method. The call returns `None` immediately after the machine starts measuring. Later, call the corresponding `collect_*_measurement()` method on the backend to retrieve and parse the results.

Poll `request_machine_status()` and wait for the `unread_data` flag to become `True` before collecting — this indicates that the machine has finished measuring and the results are ready to read.

In [9]:
# Start a multi-wavelength absorbance read without waiting
await pr.read_absorbance(
    wavelength=600,
    wavelengths=[260, 280, 450, 600],
    wait=False,
    use_new_return_type=True
)
# Returns None immediately — the machine is now measuring in the background

In [ ]:
# Poll until measurement data is available, then collect results
import asyncio

status = await clariostar_backend.request_machine_status()
while not status["unread_data"]:
    print("Measuring...")
    await asyncio.sleep(1)
    status = await clariostar_backend.request_machine_status()

# Retrieve and parse the data
results = await clariostar_backend.collect_absorbance_measurement(
    plate=plate,
    wells=plate.get_all_items(),
    wavelengths=[260, 280, 450, 600],
)

for r in results:
    print(f"Wavelength {r['wavelength']} nm -> A1 OD: {r['data'][0][0]}")

The same pattern works for fluorescence and luminescence:

| Read method | Collect method |
|-------------|---------------|
| `read_absorbance(..., wait=False)` | `clariostar_backend.collect_absorbance_measurement(plate, wells, wavelengths)` |
| `read_fluorescence(..., wait=False)` | `clariostar_backend.collect_fluorescence_measurement(plate, wells, ex_wl, em_wl)` |
| `read_luminescence(..., wait=False)` | `clariostar_backend.collect_luminescence_measurement(plate, wells)` |

```{note}
`wait=False` requires `use_new_return_type=True`. The `collect_*` methods are called directly on the backend (`clariostar_backend`), not the `PlateReader` frontend.
```

---
(reference-how-absorbance-calculation-works)=
## Reference: How Absorbance Calculation Works

The machine does not measure absorbance directly. It measures **raw detector counts** and the backend converts them to transmittance and then to OD. Understanding this pipeline helps when debugging unexpected values or when you need access to the raw data.

#### Detector channels

The CLARIOstar has four detector channels that operate simultaneously during an absorbance measurement:

| Channel | What it measures |
|---------|-----------------|
| **Chromatic 1** | Sample detector — the primary measurement at the requested wavelength |
| **Chromatic 2** | Secondary detector channel |
| **Chromatic 3** | Tertiary detector channel |
| **Reference** | Reference detector — tracks lamp intensity fluctuations across the plate |

Each channel has a calibration pair: a **high** value (100% transmission, measured through air) and a **low** value (0% transmission / dark current baseline). The high values are used in the OD conversion; the low (dark) values are embedded in the response but **not used** — see below.

#### Raw values returned by the firmware

The firmware returns four groups of values per measurement run (one per detector channel), followed by calibration data:

| Group | Count | What it is |
|-------|-------|-----------|
| **Group 0: Chromatic 1** | wells × wavelengths | Sample detector counts |
| **Group 1: Chromatic 2** | wells | Secondary detector counts |
| **Group 2: Chromatic 3** | wells | Tertiary detector counts |
| **Group 3: Reference** | wells | Reference detector counts |
| **Calibration** | 4 × (hi, lo) | One pair per channel — raw detector counts, same scale as data |

#### The conversion

The backend computes transmittance using a **reference-corrected formula (no dark subtraction)**:

**Step 1 — Normalize the sample signal:**
```
signal = sample / chromat_hi
```
This maps the sample reading onto a 0–1 scale relative to the 100% transmission calibration (measured through air). The `chromat_hi` value already includes the dark baseline, so subtracting `chromat_lo` would double-count it.

**Step 2 — Normalize the reference** (correct for lamp variation):
```
ref_norm = ref_hi / ref_well
```
This corrects for per-well lamp intensity differences. If a flash was slightly brighter for one well, both sample and reference see the same increase, and dividing cancels it out.

**Step 3 — Calculate percent transmittance:**
```
T% = signal × ref_norm × 100 = (sample / chromat_hi) × (ref_hi / ref_well) × 100
```
An empty well gives T% ≈ 100%.

**Step 4 — Convert to OD** (log transform):
```
OD = -log10(T% / 100) = -log10(signal × ref_norm)
```
This is the Beer-Lambert conversion. A blank well (T% ≈ 100) gives OD ≈ 0. A dense bacterial culture (T% ≈ 10) gives OD ≈ 1.0.

| Transmittance | OD | Meaning |
|--------------:|---:|---------|
| 100% | 0.0 | No absorption (blank / empty well) |
| 10% | 1.0 | 90% of light absorbed |
| 1% | 2.0 | 99% of light absorbed |

```{note}
The `chromat_lo` and `ref_lo` dark calibration values are embedded in every measurement response but are **not used** in the OD calculation. The `chromat_hi` calibration already includes the dark baseline (it's a raw count, not a dark-subtracted count), so subtracting `chromat_lo` would double-count the dark current. This was verified by matching all 96 wells of a test plate against OEM MARS software output to within ±0.001 OD.
```

#### Report modes

| `report=` | What you get | Description |
|-----------|-------------|:------------|
| `"OD"` (default) | Optical density values | Reference-corrected, log-transformed |
| `"transmittance"` | Percent transmittance (0–100) | Reference-corrected |
| `"raw"` | Raw detector counts + calibration data | Unprocessed firmware values |

The `"raw"` report mode returns each wavelength dict with additional keys: `references` (per-well reference counts), `chromatic_cal` (hi/lo pair for that wavelength), `reference_cal` (hi/lo pair for the reference channel), and the secondary/tertiary channel data (`chromatic2`, `chromatic3`, `chromatic2_cal`, `chromatic3_cal`). This is useful for custom calibration pipelines, quality control, or investigating detector behavior.

---
(planned-features)=
## Planned Features

The CLARIOstar firmware supports several additional features that are documented in the BMG ActiveX/DDE manual but **not yet implemented** in the PyLabRobot backend. These require USB traffic captures of the corresponding firmware byte sequences.

### Filter-Based Fluorescence

The CLARIOstar has 11 physical filter slots (4 excitation, 3 dichroic, 4 emission) alongside the LVF monochromator, sharing the same mechanical rail. Physical filters provide higher sensitivity than the monochromator and are required for certain assay types:

| Assay Type | Why Filters Are Needed |
|------------|----------------------|
| **Fluorescence Polarization (FP)** | Requires polarizer filters in excitation and emission positions |
| **HTRF / TR-FRET** | Needs dedicated time-resolved filters + dual chromatics (e.g. Fura-2) |
| **AlphaScreen / AlphaLISA** | Uses dedicated laser + specific filter sets |
| **High-sensitivity FI** | Filters give ~2× better sensitivity (0.15 pM vs 0.35 pM Fluorescein at top) |

Sensitivity comparison (per operating manual, Fluorescein, 384sv, 20 µL):

| Path | Top Reading | Bottom Reading |
|------|-----------|---------------|
| Filters | 0.15 pM (< 3 amol/well) | 1.0 pM (< 50 amol/well) |
| Monochromator | 0.35 pM (< 7 amol/well) | 3.0 pM (< 150 amol/well) |

```python
# Planned API (not yet implemented)
results = await pr.read_fluorescence(
    excitation_wavelength=485,     # for metadata only in filter mode
    emission_wavelength=528,
    focal_height=8.5,
    optic_path="filter",           # switch from monochromator to filters
    ex_filter_position=1,          # excitation filter slot 1-4
    em_filter_position=5,          # emission filter slot 5-8
    dichroic_position="A",         # dichroic mirror slot A/B/C
    use_new_return_type=True,
)
```

Filter positions correspond to the physical slots visible when opening the filter cover (see Section 6.3 of the operating manual). To check which filters are installed, use the OEM control software: **Settings → Filter → Detect all filters** (password: `bmg`).

See `CLARIOSTAR_OPTICAL_PATH_PLAN.md` for the full implementation plan including USB capture steps.

### Spectral Scanning

The CLARIOstar's monochromator hardware supports continuous wavelength scanning, where the instrument sweeps through a range of wavelengths and returns an intensity value at each step. This is useful for identifying unknown absorption/emission peaks, verifying filter selection, or characterizing new fluorophores.

```python
# Planned API (not yet implemented)
spectrum = await clariostar_backend.absorbance_scan(
    plate=plate,
    wells=[plate.get_well("A1")],
    wavelength_start=220,
    wavelength_end=750,
    wavelength_step=2,
)
# spectrum["A1"] -> [(220, 0.05), (222, 0.06), ..., (750, 0.02)]
```

Both absorbance and fluorescence spectral scans are supported by the hardware:
- **Absorbance scan**: Sweeps the monochromator across a wavelength range, measuring OD at each step. Useful for finding the absorption peak of a dye or checking sample purity (e.g. A260/A280 ratio for nucleic acids).
- **Fluorescence excitation scan**: Fixes the emission wavelength and sweeps the excitation monochromator.
- **Fluorescence emission scan**: Fixes the excitation wavelength and sweeps the emission monochromator.

### Standalone Shaking

The firmware supports a dedicated `Shake` command independent of any measurement run. This would enable:

```python
# Planned API (not yet implemented)
await clariostar_backend.shake(
    shake_type=ShakerType.ORBITAL,
    speed_rpm=300,
    duration_s=60
)
```

Per the OEM manual, standalone shaking supports:
- 5 shake types: orbital, linear, double orbital, meander corner well, orbital corner well
- Speed: 100–700 RPM (up to 1100 with high-speed shaking option)
- Duration: 1–3600 seconds
- Optional X/Y position parameters

### Idle Movement

The firmware supports an `IdleMove` command for continuous plate movement between measurements (useful during long incubation periods):

```python
# Planned API (not yet implemented)
await clariostar_backend.idle_move(mode="orbital", speed_rpm=200, duration_s=300)
await clariostar_backend.idle_move(mode="cancel")  # stop idle movement
```

Per the OEM manual, 7 modes are available including linear corner-to-corner movement, incubation position waiting, and various shaking patterns with configurable on/off cycling.

### Injector System

The CLARIOstar supports up to two injector pumps (`Pump1` / `Pump2`) for dispensing reagents during kinetic reads. Not yet implemented.

```{note}
To help implement any of these features, capture USB traffic while executing the corresponding command in the BMG OEM control software and open an issue on the [PyLabRobot GitHub](https://github.com/PyLabRobot/pylabrobot/issues).
```

---
## Closing Connection

In [ ]:
pr.unassign_child_resource(plate)

In [ ]:
await pr.stop()

This closes the FTDI connection. After calling `stop()`, you must call `setup()` again before using the plate reader.